In [1]:
from jarvis.utils.general import gpus
gpus.autoselect()

import numpy as np, pandas as pd, tensorflow as tf, os, time
from pathlib import Path
from IPython.display import clear_output, HTML, Javascript, display
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, accuracy_score, roc_auc_score, average_precision_score

import sys  
sys.path.append('/home/jjlou/Jerry/jerry_packages')
from jerry_utils import restart_kernel, load_dataset, save_dataset
import jerry_losses, jerry_metrics 

[ 2023-11-24 21:20:54 ] CUDA_VISIBLE_DEVICES automatically set to: 2           


In [2]:
name = 'Unfrozen_EfficientNetV2L_v0.2'
root = '/home/jjlou/Jerry/wsi-arterio/arteriosclerotic_vessel_detection_and_fine_segmentation/Arteriolosclerosis_classification/data'

In [3]:
Valid0 = ['D-436','D-870', 'D-343']
Valid1 = ['D-297', 'D-916', 'UCI-15-12']
Valid2 = ['D-322', 'D-562']

Valid = [Valid0, Valid1, Valid2]

In [4]:
#tf.numpy_function creates output with unknown shape.. so need to reshape it
def reshape(image, labels):
    labels = tf.reshape(labels, [1,])
    return tf.cast(image, 'uint8'), tf.cast(labels, 'uint8')

In [5]:
for v in Valid:
    v_path = f"{root}/K{Valid.index(v)}"
    train_path = f"{v_path}/Train_K{Valid.index(v)}_Color.BasicMorph.Aug"  
    valid_path = f"{v_path}/Valid_K{Valid.index(v)}_raw"
    model_path = f'{v_path}/models/{name}_Color.BasicMorph.Aug_K{Valid.index(v)}.hdf5'
    save_path = f'{v_path}/models/{name}_Color.BasicMorph.Aug_K{Valid.index(v)}'
    
    if os.path.exists(f'{save_path}_valid.csv'): 
        continue
    else:
        # load data
        train = load_dataset(train_path) 
        valid = load_dataset(valid_path)
        
        train = train.map(lambda i,m: reshape(i,m), num_parallel_calls=tf.data.AUTOTUNE)
        valid = valid.batch(1)
        
        # load model
        model = tf.keras.models.load_model(model_path)
        
        if not os.path.exists(f'{save_path}_train.npz'): 
            true_list = []
            pred_list = []
            for x, y in train:
                logits = model.predict(x) 
                pred = tf.cast(np.squeeze(np.argmax(logits, axis=-1)), tf.uint8)
                true = tf.squeeze(y)
                pred_list.append(pred)
                true_list.append(true)    
            pred_np = np.stack(pred_list)
            true_np = np.stack(true_list)
            np.savez(f'{save_path}_train.npz', pred=pred_np, true=true_np)
            restart_kernel()
            time.sleep(10)
            
        elif not os.path.exists(f'{save_path}_valid.npz'): 
            true_list = []
            pred_list = []
            for x, y in valid:
                logits = model.predict(x) 
                pred = tf.cast(np.squeeze(np.argmax(logits, axis=-1)), tf.uint8)
                true = tf.squeeze(y)
                pred_list.append(pred)
                true_list.append(true)
            pred_np = np.stack(pred_list)
            true_np = np.stack(true_list)
            np.savez(f'{save_path}_valid.npz', pred=pred_np, true=true_np)   
            restart_kernel()
            time.sleep(10)
            
        else:
            continue

TypeError: 'str' object does not support item assignment